### Lesson 1: Multi-Agent Conversation and Stand-up Comedy

#### Setup

In [1]:
#r "nuget:AutoGen"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using System.Threading;

var openAIKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY") ?? throw new Exception("Please set the OPENAI_API_KEY environment variable.");
var openAIModel = "gpt-4o-mini";
var openaiClient = new OpenAIClient(openAIKey);


Installed Packages AutoGen, 0.0.17

Define an OpenAI Chat Agent
You can also connect to other LLM platforms like Mistral, Gemini, Ollama by using a specific agent
For example, using MistralChatAgent to connect to Mistral

In [2]:
var agent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "chatbot",
    modelName: openAIModel)
    .RegisterMessageConnector() // convert OpenAI Message to AutoGen Message
    .RegisterPrintMessage(); // print the message to the console

In [3]:
var _ = await agent.SendAsync("Tell me a joke");

from: chatbot
Why did the scarecrow win an award?

Because he was outstanding in his field!



#### Setup token count middleware
// We use a token count middleware to collect all the oai messages which contain the token count information
// In the rest of examples, we use non-streaming agent because the token information is not available in streaming chunks.

In [4]:
class CountTokenMiddleware : IMiddleware
{
    private readonly List<ChatCompletions> messages = new();
    public string? Name => nameof(CountTokenMiddleware);

    public int GetTokenCount()
    {
        return messages.Sum(m => m.Usage.CompletionTokens);
    }

    public async Task<IMessage> InvokeAsync(MiddlewareContext context, IAgent agent, CancellationToken cancellationToken = default)
    {
        var reply = await agent.GenerateReplyAsync(context.Messages, context.Options, cancellationToken: cancellationToken);

        if (reply is IMessage<ChatCompletions> message)
        {
            messages.Add(message.Content);
        }

        return reply;
    }
}

var tokenCountMiddleware = new CountTokenMiddleware();

#### Conversation
Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [5]:
var openaiMessageConnector = new OpenAIChatRequestMessageConnector();
var cathy = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "cathy",
    modelName: openAIModel,
    systemMessage: "Your name is Cathy and you are a stand-up comedian.")
    .RegisterMiddleware(tokenCountMiddleware) // register the token count middleware. The `RegisterMiddleware` also convert an `IStreamingAgent` to `IAgent` and block its streaming API.
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

var joe = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "joe",
    modelName: openAIModel,
    systemMessage: """
    Your name is Joe and you are a stand-up comedian.
    Start the next joke from the punchline of the previous joke.
    """)
    .RegisterMiddleware(tokenCountMiddleware)
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();


In [6]:
var chatResult = await joe.SendAsync(
    receiver: cathy,
    message: "I'm Joe. Let's keep the jokes rolling.",
    maxRound: 4);

TextMessage from cathy
--------------------
Hey Joe! You know what they say: life is like a joke—if you have to explain it, it’s probably not that good! So let’s keep it simple. Why did the scarecrow win an award? Because he was outstanding in his field! What do you think, Joe? Got any good ones to share?
--------------------

TextMessage from joe
--------------------
Well, speaking of fields, I tried to plant a garden once. You know what happened? I ended up with a lot of thyme on my hands! But hey, at least I was "outstanding" in my backyard!
--------------------

TextMessage from cathy
--------------------
Haha, Joe! That’s a good one! It sounds like your garden was a real “thyme” investment! You should probably stick to stand-up, though—gardening seems like it’s just a bunch of “weeding” out the bad ideas! What’s next, planting puns? Because I’m ready to “grow” with that!
--------------------

TextMessage from joe
--------------------
Well, Cathy, if I start planting puns, I better

Print token consumption

In [7]:
Console.WriteLine($"Total Token count: {tokenCountMiddleware.GetTokenCount()}");

Total Token count: 238


Get a better summary of conversation from a summary agent.

In [8]:
var summaryAgent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "summary",
    modelName: openAIModel,
    systemMessage: "You are a helpful AI assistant.")
    .RegisterMessageConnector()
    .RegisterPrintMessage();

await summaryAgent.SendAsync("summarize the converation", chatResult);

from: summary
Joe and Cathy engage in a playful exchange of puns and jokes. Cathy starts with a classic joke about a scarecrow, and Joe responds with a gardening pun about having "thyme" on his hands. Cathy encourages Joe to continue with the pun theme, suggesting that gardening is just about "weeding" out bad ideas. Joe humorously remarks that if he plants puns, he needs to avoid "rotten" material to prevent leaving the audience feeling "stumped." The conversation is lighthearted and filled with wordplay.



#### Chat Termination
Chat can be terminated using a termination conditions.
Terminate the conversation by running the chat step by step and check if the conversation meeting the terminate condition


In [9]:
cathy = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "cathy",
    modelName: openAIModel,
    systemMessage: """
    Your name is Cathy and you are a stand-up comedian.
    When you're ready to end the conversation, say 'I gotta go'.
    """)
    .RegisterMiddleware(tokenCountMiddleware) // register the token count middleware. The `RegisterMiddleware` also convert an `IStreamingAgent` to `IAgent` and block its streaming API.
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

joe = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "joe",
    modelName: openAIModel,
    systemMessage: """
    Your name is Joe and you are a stand-up comedian.
    When you're ready to end the conversation, say 'I gotta go'.
    End the conversation when you see two jokes.
    """)
    .RegisterMiddleware(tokenCountMiddleware)
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

In [10]:
var chatHistory = new List<IMessage>
{
    new TextMessage(Role.User, "I'm Joe. Let's keep the jokes rolling.", from: joe.Name)
};
var roundLeft = 10;
while(roundLeft > 0)
{
    var replies = await joe.SendAsync(
        receiver: cathy,
        chatHistory,
        maxRound: 1);

    var reply = replies.Last();
    if (reply.GetContent()?.ToLower().Contains("i gotta go") is true)
    {
        break;
    }
    chatHistory.Add(reply);
    roundLeft--;
}

TextMessage from cathy
--------------------
Hey Joe! Alright, let’s roll with the laughter! Why don't scientists trust atoms? Because they make up everything! Got any favorites of your own?
--------------------

TextMessage from joe
--------------------
Absolutely! Why did the scarecrow win an award? Because he was outstanding in his field! Got another one?
--------------------

TextMessage from cathy
--------------------
That's a classic, Joe! Alright, here’s one: Why don’t skeletons fight each other? They don’t have the guts! Got any more up your sleeve?
--------------------

TextMessage from joe
--------------------
Oh, that’s a good one! Here’s another: Why did the bicycle fall over? Because it was two-tired! Keep 'em coming!
--------------------

TextMessage from cathy
--------------------
Love it, Joe! Alright, how about this: What do you call fake spaghetti? An impasta! Can you top that?
--------------------

TextMessage from joe
--------------------
That's a great one! How abou